# Diagnosis based on ECG data

- - - 

* Diagnosis using collected ECG data
* Currently total 42 subjects
* 3 classes (DEP, SUI, NOR)

- - -

In [ ]:
# importing required components 
import os
import time
import json
import urllib
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
from pdf2image import convert_from_path

## Basic data check

In [ ]:
ecg_df = pd.read_csv('E:/RESEARCH/Datasets/wearable/AI_coded_1.csv', sep=',')

* 1: depression, 2: suicidality, 3: normal

In [ ]:
ecg_df['class'] = ecg_df['class'].astype("category")
ecg_df['sub'] = ecg_df['sub'].astype("category")

In [ ]:
depression = ecg_df[ecg_df['class']==1]
print((depression.index)+1)

In [ ]:
suicidal = ecg_df[ecg_df['class']==2]
print((suicidal.index)+1)

In [ ]:
normal = ecg_df[ecg_df['class']==3]
print((normal.index)+1)

- - -

* Data handling

In [ ]:
## Checking file path and names
file_path = "E:/RESEARCH/Datasets/wearable/ECG/test_0420/dep/"
file_names = os.listdir(file_path)

In [ ]:
## Changing file names to 1, 2, ...
i = 1
for name in file_names:
    src = os.path.join(file_path, name)
    dst = str(i) + '.png'
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

In [ ]:
## Converting pdf file into png file
for name in file_names:
    pages = convert_from_path(file_path + name, poppler_path="E:/RESEARCH/Datasets/wearable/ECG/poppler/Library/bin")
    
    for page in pages:
        page.save(file_path + name + '.png', "PNG")

In [ ]:
## Data Crop (deleting patients' information)
# original png size = 2200 x 1700
# leaving the important part only (only the ecg data part)
left = 100
top = 450
right = 2100
bottom = 1300

for name in file_names:
    im = Image.open(file_path + name)
    imc = im.crop((left, top, right, bottom))
    imc.save(file_path+name+'.png')

- - -